-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

<i18n value="a209ac48-08a6-4b89-b728-084a515fd335"/>



# Manipulating Tables with Delta Lake

This notebook provides a hands-on review of some of the basic functionality of Delta Lake.

## Learning Objectives
By the end of this lab, you should be able to:
- Execute standard operations to create and manipulate Delta Lake tables, including:
  - **`CREATE TABLE`**
  - **`INSERT INTO`**
  - **`SELECT FROM`**
  - **`UPDATE`**
  - **`DELETE`**
  - **`MERGE`**
  - **`DROP TABLE`**

<i18n value="6582dbcd-72c7-496b-adbd-23aef98e20e9"/>



## Setup
Run the following script to setup necessary variables and clear out past runs of this notebook. Note that re-executing this cell will allow you to start the lab over.

In [0]:
%run ../Includes/Classroom-Setup-02.2L

<i18n value="0607f2ed-cfe6-4a38-baa4-e6754ec1c664"/>



## Create a Table

In this notebook, we'll be creating a table to track our bean collection.

Use the cell below to create a managed Delta Lake table named **`beans`**.

Provide the following schema:

| Field Name | Field type |
| --- | --- |
| name | STRING |
| color | STRING |
| grams | FLOAT |
| delicious | BOOLEAN |

In [0]:
%sql
-- ANSWER
CREATE TABLE beans 
(name STRING, color STRING, grams FLOAT, delicious BOOLEAN); 

<i18n value="2167d7d7-93d1-4704-a7cb-a0335eaf8da7"/>



**NOTE**: We'll use Python to run checks occasionally throughout the lab. The following cell will return as error with a message on what needs to change if you have not followed instructions. No output from cell execution means that you have completed this step.

In [0]:
%python
assert spark.table("beans"), "Table named `beans` does not exist"
assert spark.table("beans").columns == ["name", "color", "grams", "delicious"], "Please name the columns in the order provided above"
assert spark.table("beans").dtypes == [("name", "string"), ("color", "string"), ("grams", "float"), ("delicious", "boolean")], "Please make sure the column types are identical to those provided above"

<i18n value="89004ef0-db16-474b-8cce-eff85c225a65"/>


## Insert Data

Run the following cell to insert three rows into the table.

In [0]:
%sql
INSERT INTO beans VALUES
("black", "black", 500, true),
("lentils", "brown", 1000, true),
("jelly", "rainbow", 42.5, false)

num_affected_rows,num_inserted_rows
3,3


<i18n value="48d649a1-cde1-491f-a90d-95d2e336e140"/>



Manually review the table contents to ensure data was written as expected.

In [0]:
%sql
-- ANSWER
SELECT * FROM beans

name,color,grams,delicious
black,black,500.0,true
lentils,brown,1000.0,true
jelly,rainbow,42.5,false


<i18n value="f0406eef-6973-47c9-8f89-a667c53cfea7"/>



Insert the additional records provided below. Make sure you execute this as a single transaction.

In [0]:
%sql
-- ANSWER
INSERT INTO beans VALUES
('pinto', 'brown', 1.5, true),
('green', 'green', 178.3, true),
('beanbag chair', 'white', 40000, false)

num_affected_rows,num_inserted_rows
3,3


<i18n value="e1764f9f-8052-47bb-a862-b52ca438378a"/>



Run the cell below to confirm the data is in the proper state.

In [0]:
%python
assert spark.table("beans").count() == 6, "The table should have 6 records"
assert spark.conf.get("spark.databricks.delta.lastCommitVersionInSession") == "2", "Only 3 commits should have been made to the table"
assert set(row["name"] for row in spark.table("beans").select("name").collect()) == {'beanbag chair', 'black', 'green', 'jelly', 'lentils', 'pinto'}, "Make sure you have not modified the data provided"

<i18n value="e38adafa-bd10-4191-9c69-e6a4363532ec"/>



## Update Records

A friend is reviewing your inventory of beans. After much debate, you agree that jelly beans are delicious.

Run the following cell to update this record.

In [0]:
%sql
UPDATE beans
SET delicious = true
WHERE name = "jelly"

num_affected_rows
1


<i18n value="d8637fab-6d23-4458-bc08-ff777021e30c"/>



You realize that you've accidentally entered the weight of your pinto beans incorrectly.

Update the **`grams`** column for this record to the correct weight of 1500.

In [0]:
%sql
-- ANSWER
UPDATE beans
SET grams = 1500
WHERE name = 'pinto'

num_affected_rows
1


<i18n value="954bf892-4db6-4b25-9a9a-83b0f6ecc123"/>



Run the cell below to confirm this has completed properly.

In [0]:
%python
assert spark.table("beans").filter("name='pinto'").count() == 1, "There should only be 1 entry for pinto beans"
row = spark.table("beans").filter("name='pinto'").first()
assert row["color"] == "brown", "The pinto bean should be labeled as the color brown"
assert row["grams"] == 1500, "Make sure you correctly specified the `grams` as 1500"
assert row["delicious"] == True, "The pinto bean is a delicious bean"

<i18n value="f36d551a-f588-43e4-84a2-6aa49a420c04"/>



## Delete Records

You've decided that you only want to keep track of delicious beans.

Execute a query to drop all beans that are not delicious.

In [0]:
%sql
-- ANSWER
DELETE FROM beans
WHERE delicious = false

num_affected_rows
1


<i18n value="1c8d924c-3e97-49a0-b5e4-0378c5acd3c8"/>



Run the following cell to confirm this operation was successful.

In [0]:
%python
assert spark.table("beans").filter("delicious=true").count() == 5, "There should be 5 delicious beans in your table"
assert spark.table("beans").filter("name='beanbag chair'").count() == 0, "Make sure your logic deletes non-delicious beans"

<i18n value="903473f1-ddca-41ea-ae2f-dc2fac64936e"/>



## Using Merge to Upsert Records

Your friend gives you some new beans. The cell below registers these as a temporary view.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW new_beans(name, color, grams, delicious) AS VALUES
('black', 'black', 60.5, true),
('lentils', 'green', 500, true),
('kidney', 'red', 387.2, true),
('castor', 'brown', 25, false);

SELECT * FROM new_beans

name,color,grams,delicious
black,black,60.5,true
lentils,green,500.0,true
kidney,red,387.2,true
castor,brown,25.0,false


<i18n value="58d50e50-65f1-403b-b74e-1143cde49356"/>



In the cell below, use the above view to write a merge statement to update and insert new records to your **`beans`** table as one transaction.

Make sure your logic:
- Matches beans by name **and** color
- Updates existing beans by adding the new weight to the existing weight
- Inserts new beans only if they are delicious

In [0]:
%sql
-- ANSWER
MERGE INTO beans a
USING new_beans b
ON a.name=b.name AND a.color = b.color
WHEN MATCHED THEN
  UPDATE SET grams = a.grams + b.grams
WHEN NOT MATCHED AND b.delicious = true THEN
  INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
3,1,0,2


<i18n value="9fbb65eb-9119-482f-ab77-35e11af5fb24"/>



Run the cell below to check your work.

In [0]:
%python
import pyspark.sql.functions as F
last_version = spark.sql("DESCRIBE HISTORY beans").orderBy(F.col("version").desc()).first()

assert last_version["operation"] == "MERGE", "Transaction should be completed as a merge"

metrics = last_version["operationMetrics"]
assert metrics["numOutputRows"] == "5", "Make sure you only insert delicious beans"
assert metrics["numTargetRowsUpdated"] == "1", "Make sure you match on name and color"
assert metrics["numTargetRowsInserted"] == "2", "Make sure you insert newly collected beans"
assert metrics["numTargetRowsDeleted"] == "0", "No rows should be deleted by this operation"

<i18n value="4a668d7c-e16b-4061-a5b7-1ec732236308"/>



## Dropping Tables

When working with managed Delta Lake tables, dropping a table results in permanently deleting access to the table and all underlying data files.

**NOTE**: Later in the course, we'll learn about external tables, which approach Delta Lake tables as a collection of files and have different persistence guarantees.

In the cell below, write a query to drop the **`beans`** table.

In [0]:
%sql
-- ANSWER
DROP TABLE beans

<i18n value="4cc5c126-5e56-423e-a814-f6c422312802"/>



Run the cell below to assert that your table no longer exists.

In [0]:
%python
assert spark.sql("SHOW TABLES LIKE 'beans'").collect() == [], "Confirm that you have dropped the `beans` table from your current database"

---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
<command-4094000743660281> in <cell line: 1>()
----> 1 assert spark.sql("SHOW TABLES LIKE 'beans'").collect() == [], "Confirm that you have dropped the `beans` table from your current database"

AssertionError: Confirm that you have dropped the `beans` table from your current database

<i18n value="f4d330e3-dc40-4b6e-9911-34902bab22ae"/>



## Wrapping Up

By completing this lab, you should now feel comfortable:
* Completing standard Delta Lake table creation and data manipulation commands

<i18n value="d59f9828-9b13-4e0e-ae98-7e852cd32198"/>



Run the following cell to delete the tables and files associated with this lesson.

In [0]:
%python
DA.cleanup()

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>